In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import glob
from collections import Counter


def stat_parity_rank_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns]==0).sum(axis=0)/g_df.shape[0]
    return abs(data['male'] - data['female'])

def preprocess_df(df):
    if df.columns[1:][0]=="epoch_1":
        li = ["ids"] 
        for i in range(0,len(df.columns[1:])):
            li.append("epoch_"+str(i))
        df.columns = li
        return df
    else:
        return df
def acc_by_gender(df):
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = g_df['rank_by_id'].sum(axis=0)/g_df.shape[0]
    return data['male'],data['female']

def stat_parity_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns]==0).sum(axis=0)/g_df.shape[0]
    return abs(data['male'] - data['female'])

def stat_parity_ratio_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns]==0).sum(axis=0)/g_df.shape[0]
    return np.abs(1-data['male']/data['female'])

def stat_parity_ratio_rank_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns]==0).sum(axis=0)/g_df.shape[0]
    return np.abs(1-data['male']/data['female'])

def ratio_errors_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns]!=0).sum(axis=0)/g_df.shape[0]
    return np.abs(1-data['male']/data['female'])

def stat_parity_from_rank_ratio_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns] == 0).sum(axis=0)/g_df.shape[0]
    return np.abs(1-data['male']/data['female'])

def rank_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns]).sum(axis=0)/g_df.shape[0]
    return abs(data['male']-data['female'])

def rank_ratio_func(df, epoch_columns):
    # calculate the violation of statistical parity
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = (g_df[epoch_columns]).sum(axis=0)/g_df.shape[0]
    return np.abs(1-data['male']/data['female'])

def acc_func(df, epoch_columns):
    # calculate the accuracy 
    return df[epoch_columns].sum(axis=0)/df.shape[0]

def acc_from_rank_func(df, epoch_columns):
    # calculate the accuracy 
    return (df[epoch_columns] == 0).sum(axis=0)/df.shape[0]

def acc_by_gender(df):
    data = {}
    for g,g_df in df.groupby('gender_expression'):
        data[g] = g_df['rank_by_id'].sum(axis=0)/g_df.shape[0]
    return data['male']
def acc_from_rank_func_male(df, epoch_columns):
    # calculate the accuracy 
    return (df[epoch_columns] == 0).sum(axis=0)/df.shape[0]

def err_from_rank_func(df, epoch_columns):
    # calculate the accuracy 
    return (df[epoch_columns] != 0).sum(axis=0)/df.shape[0]

def get_name_details(f):
    head_id = -4 if 'cosine' in f else -2
    opt_id = -3 if 'cosine' in f else -1
    y = os.path.splitext(os.path.basename(f))[0]
    experiment = y.replace('config_','')
    head = experiment.split('_')[head_id]
    opt = experiment.split('_')[opt_id]
    model = '_'.join(experiment.split('_')[:head_id])
    return experiment, model, head, opt


def analyze_files(files, metadata, ratio=False, error=False, epochs=None):
    acc_df = pd.DataFrame(columns=['epoch_'+str(e) for e in range(100)])
    acc_disp_df = pd.DataFrame(columns=['epoch_'+str(e) for e in range(100)])
    for f in files:
        try:
            df = pd.read_csv(f)
            df  = preprocess_df(df)
        except:
            continue
        epoch_columns = df.drop('ids',axis=1).columns
        df = metadata.merge(df)
        num_epochs = len(epoch_columns)
        df[epoch_columns] = df[epoch_columns].apply(lambda x: x == df['label'])
        acc = acc_func(df, epoch_columns)
        experiment = get_name_details(f)[0]
        acc_df.loc[experiment] = acc
        if ratio:
            if error:
                acc_disp = ratio_errors_func(df, epoch_columns)
            else:
                acc_disp = stat_parity_ratio_func(df, epoch_columns)
        else:
            acc_disp = stat_parity_func(df, epoch_columns)
        acc_disp_df.loc[experiment] = acc_disp    
    return acc_df, acc_disp_df

def analyze_rank_files_np(files, metadata, ratio=False, error=False, epochs=None): 
    if epochs is None:
        epochs = ['epoch_'+str(e) for e in range(101)]
        
    acc_df = pd.DataFrame(columns=epochs)
    acc_ratio_df = pd.DataFrame(columns=epochs)
    rank_df = pd.DataFrame(columns=epochs)
    male =[]
    female = []
    for f in files:
        try:
            df = pd.read_csv(f)
            df  = preprocess_df(df)
        except:
            print(f)
            print("here")
            continue
        epoch_columns = list(set(df.columns).intersection(epochs))
        df = metadata.merge(df)
        num_epochs = len(epoch_columns)
        if error:
            acc = err_from_rank_func(df, epoch_columns)
        else:
            acc = acc_from_rank_func(df, epoch_columns)
        experiment = f.split('/')[0]
        acc_df.loc[experiment] = acc
        
        if ratio:
            if error:
                acc_disp = ratio_errors_func(df, epoch_columns)
            else:
                acc_disp = stat_parity_ratio_rank_func(df, epoch_columns)
        else:
            acc_disp = stat_parity_rank_func(df, epoch_columns)
        acc_ratio_df.loc[experiment] = acc_disp 
        
        if ratio:
            rank_ratio = rank_ratio_func(df, epoch_columns)
        else:
            rank_ratio = rank_func(df, epoch_columns)
        rank_df.loc[experiment] = rank_ratio  
    return acc_df, acc_ratio_df, rank_df

def analyze_rank_files(files, metadata, ratio=False, error=False, epochs=None): 
    acc_df, acc_ratio_df, rank_df = analyze_rank_files_np(files, metadata, ratio=ratio, error=error, epochs=epochs)
    return prepare(acc_df), prepare(acc_ratio_df), prepare(rank_df)

def plot_df(acc_df, acc_disp_df, rank_df = None, title = ''):
    def prepare(df):
        # dataframe of a long format
        df = pd.melt(df.reset_index(), id_vars='index')
        df = df.rename(columns={'variable':'epoch'})
        df = df.rename(columns={'value':'Accuracy'})
        df.epoch = df.epoch.apply(lambda x: int(x.split('_')[1]))
        return df
    acc_df = prepare(acc_df)
    acc_disp_df = prepare(acc_disp_df)

    # plotly express
    acc_df['measurement'] = 'Accuracy'
    acc_disp_df['measurement'] = 'Disparity'

    df = acc_df.append(acc_disp_df)

    if rank_df is not None:
        rank_df = prepare(rank_df)
        rank_df['measurement'] = 'Rank'
        df = df.append(rank_df)
        
    df = df.dropna()

    fig = px.line(df, x='epoch', y='Accuracy', color='index', facet_row='measurement', title=title)
    fig.update_yaxes(matches=None)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_layout(yaxis_title="Disparity")

    fig.show()
    
    
def whatIsPareto(df, x_inc=False, y_inc=False):
    isPareto = np.zeros(df.shape[0])
    i = 0
    for _, (c1,c2) in df.iterrows():
        tmp = np.delete(np.array(df), (i), axis=0)
        if x_inc: # is a larger x better?
            if y_inc: # is a larger y better?
                b = np.any(np.apply_along_axis(lambda x: x[0]>c1 and x[1]>c2, 1, tmp))
            else: # is a smaller y better?
                b = np.any(np.apply_along_axis(lambda x: x[0]>c1 and x[1]<c2, 1, tmp))
        else: # is a smaller x better?
            if y_inc: # is a larger y better?
                b = np.any(np.apply_along_axis(lambda x: x[0]<c1 and x[1]>c2, 1, tmp))
            else: # is a smaller y better?
                b = np.any(np.apply_along_axis(lambda x: x[0]<c1 and x[1]<c2, 1, tmp))
        if not b:
            isPareto[i] = 1
        i+=1
    return isPareto

def preparePareto(df, x_inc=False, y_inc=False):
    
    isPareto = whatIsPareto(df, x_inc=x_inc, y_inc=y_inc)
    tmp = df[isPareto == 1]
    
    tmp = tmp.sort_values(df.columns[0])
    return tmp

def prepare(df):
    # dataframe of a long format
    #print(df.head())
    df = pd.melt(df.reset_index(), id_vars='index')
    df = df.rename(columns={'variable':'epoch'})
    df = df.rename(columns={'value':'Metric'})
    df.epoch = df.epoch.apply(lambda x: int(x.split('_')[1]))
    return df

def merge(df1, df2):
    df = df1.merge(df2, on=["index","epoch"])
    df = df.rename(columns={'Metric_x':'Accuracy'})
    df = df.rename(columns={'Metric_y':'Disparity'})
    return df

def drop_models(df_list, models):
    # remove rows with model names in models from each df in the df_list
    out_list = []
    for df in df_list:
        out_list += [df[~df['index'].isin(models)]]
    return out_list


def find_yaml_folder(yaml):
    '''
    given a yaml string file like:
         'config_inception_resnet_v2_CosFace_RMSProp.yaml'
    return the corresponding folder for this experiment:
         './Phase1B/inception_resnet_v2_CosFace_RMSProp'
    if it does not exist, return ''
    '''
    experiment_name = yaml.replace('config_','').replace('.yaml','')

    experiment_folders = glob.glob('/cmlscratch/sdooley1/merge_timm/FR-NAS/Checkpoints/Phase1B/*') + glob.glob('/cmlscratch/sdooley1/merge_timm/FR-NAS/Checkpoints/timm_explore_few_epochs/*')
    where = [get_name_details(experiment_name)[0].lower() == get_name_details(x)[0].lower() for x in experiment_folders]
    yaml_folder = ''
    if any(where):
        yaml_folder = experiment_folders[np.where(where)[0][0]]
    return yaml_folder

def get_finished_models_Phase1B():
    '''
    Return a list of those models which we are including in Phase1B
    '''
    finished = []
    for yaml_orig in glob.glob('/cmlscratch/sdooley1/merge_timm/FR-NAS/configs/**/*.yaml') + glob.glob('/cmlscratch/sdooley1/merge_timm/FR-NAS/configs_multi/**/*.yaml'):
        yaml = os.path.basename(yaml_orig)
        yaml_folder = find_yaml_folder(yaml)
        if yaml_folder:
            finished += [yaml]
    cn = Counter([get_name_details(x)[1] for x in finished])
    final_models = [k for k,v in cn.items() if v>=6]
    final_models.sort()

    return final_models


def get_pareto_hps_head_opt(stable_df, col='Accuracy'):
    row = []
    for opt in ['adamw', 'sgd']:
        for head in ['ArcFace','CosFace','MagFace']:
            df = stable_df
            df = df[(df['opt'] == opt) & (df['head'] == head)]
            ind = whatIsPareto(df[[col,'Disparity']], True, False).astype(bool)
            out = df[ind].dropna().sort_values(col, ascending=False)
            m = out['model'].to_string(header=False,index=False).split('\n')
            row += ['\n'.join(list(np.unique([x.strip() for x in m])))]
    return row

def get_pareto_hps_opt(stable_df, col='Accuracy'):
    row = []
    for opt in ['adamw', 'sgd']:
            df = stable_df
            df = df[(df['opt'] == opt)]
            ind = whatIsPareto(df[[col,'Disparity']], True, False).astype(bool)
            out = df[ind].dropna().sort_values(col, ascending=False)
            m = out['model'].to_string(header=False,index=False).split('\n')
            row += ['\n'.join(list(np.unique([x.strip() for x in m])))]
    return row

def get_pareto_hps_head(stable_df, col='Accuracy'):
    row = []
    for head in ['ArcFace', 'CosFace', 'MagFace']:
            df = stable_df
            df = df[(df['head'] == head)]
            ind = whatIsPareto(df[[col,'Disparity']], True, False).astype(bool)
            out = df[ind].dropna().sort_values(col, ascending=False)
            m = out['model'].to_string(header=False,index=False).split('\n')
            row += ['\n'.join(list(np.unique([x.strip() for x in m])))]
    return row

def anova_hp_accuracy(df, col = 'Accuracy'):
    df['model'] = df['index'].apply(lambda x: get_name_details(x.replace('_rank_by_id_val',''))[1])
    df['head'] = df['index'].apply(lambda x: get_name_details(x.replace('_rank_by_id_val',''))[2])
    df['opt'] = df['index'].apply(lambda x: get_name_details(x.replace('_rank_by_id_val',''))[3].lower())
    df = df.merge(meta, left_on='model', right_on='model_name')
    df.fillna('0',inplace=True)
    df[col] = df[col].astype(float)

    lm = ols(col+' ~ head + opt', data=df).fit() # fitting the model
    
    print(sm.stats.anova_lm(lm))
    tukey_head = pairwise_tukeyhsd(endog=df[col],
                              groups=df['head'],
                              alpha=0.05)
    print(tukey_head)
    tukey_opt = pairwise_tukeyhsd(endog=df[col],
                              groups=df['opt'],
                              alpha=0.05)
    print(tukey_opt)
    
    return sm.stats.anova_lm(lm), tukey_head, tukey_opt

def anova_hp_disp(df, col = 'Accuracy'):
    df['model'] = df['index'].apply(lambda x: get_name_details(x.replace('_rank_by_id_val',''))[1])
    df['head'] = df['index'].apply(lambda x: get_name_details(x.replace('_rank_by_id_val',''))[2])
    df['opt'] = df['index'].apply(lambda x: get_name_details(x.replace('_rank_by_id_val',''))[3].lower())
    df = df.merge(meta, left_on='model', right_on='model_name')
    df.fillna('0',inplace=True)
    df['Disparity'] = df['Disparity'].astype(float)

    lm = ols('Disparity ~ head + opt', data=df).fit() # fitting the model
    
    print(sm.stats.anova_lm(lm))
    tukey_head = pairwise_tukeyhsd(endog=df['Disparity'],
                              groups=df['head'],
                              alpha=0.05)
    print(tukey_head)
    tukey_opt = pairwise_tukeyhsd(endog=df['Disparity'],
                              groups=df['opt'],
                              alpha=0.05)
    print(tukey_opt)
    
    return sm.stats.anova_lm(lm), tukey_head, tukey_opt

In [21]:
#Plot male v/s female accuracy
import os
os.chdir('/work/dlclarge2/sukthank-ZCP_Competition/FRNAS_latest/FR-NAS')
metadata = pd.read_csv('val_identities_gender-expression_seed_222.csv')
os.chdir('Checkpoints_scratch/')
import pandas as pd
import numpy as np
import os
import glob
phase1bii_configs = glob.glob('*/*_rank_by_id_val.csv')
models = list(set([x.split('/')[1] for x in phase1bii_configs]))
rank_files = glob.glob('*/*_rank_by_id_val.csv')
#print(kacc_files)


epochs = ['epoch_'+str(i) for i in [99]]
acc_df, acc_disp_df, rank_df = analyze_rank_files(rank_files, metadata, epochs=epochs)
acc_df, acc_disp_df, rank_df = analyze_rank_files(rank_files, metadata, epochs=epochs)
_, acc_disp_ratio_df, rank_ratio_df = analyze_rank_files(rank_files, metadata, ratio=True, epochs=epochs)
err_df, error_ratio_df, _ = analyze_rank_files(rank_files, metadata, ratio=True, error=True, epochs=epochs)

In [22]:

df_rank = merge(acc_df, rank_df)
#print(df.tail(400))
fig = px.scatter(df_rank, x='Accuracy', y='Disparity', color='index')
fig.update_layout(
    yaxis_title="Disparity |Rank[male]-Rank[female]|",
)
fig.add_shape(type='line',
                x0=0,y0=0,x1=1,y1=0,
                line=dict(color='Red',),
                xref='x',yref='y'
)
p = np.array(preparePareto(df_rank[['Accuracy','Disparity']], True, False).dropna())
for x, y in zip(p[:-1], p[1:]):
    fig.add_shape(type='line',
                x0=x[0],y0=x[1],x1=y[0],y1=y[1],
                line=dict(color='gray',),line_dash='dash',
                xref='x',yref='y')
fig.write_html("pareto_rank_disp.html")
fig.show()

In [27]:
from scipy.stats import sem
seed =4
df_rank["Accuracy"] = 1-df_rank["Accuracy"]
mobnet = df_rank[df_rank["index"].str.contains(r'mobilenetv3_large_100_CosFace_SGD_0.1_cosine*')]
mobnet = mobnet[mobnet["epoch"]==99].dropna().tail(seed)
tnt = df_rank[df_rank["index"].str.contains(r'tnt_s_patch16_224_CosFace_AdamW*')]
tnt = tnt[tnt["epoch"]==99].dropna().tail(seed)
rexnet = df_rank[df_rank["index"].str.contains(r'rexnet_200_MagFace_SGD_0.1_cosine*')]
rexnet = rexnet[rexnet["epoch"]==99].dropna().tail(seed)
inception = df_rank[df_rank["index"].str.contains(r'inception_resnet_v2_CosFace_AdamW_0.001_cosine*')]
inception=inception[inception["epoch"]==99].dropna().tail(seed)
ese = df_rank[df_rank["index"].str.contains(r'ese_vovnet39b_CosFace_AdamW_0.001_cosine*')]
ese=ese[ese["epoch"]==99].tail(4).dropna()
densenet161 = df_rank[df_rank["index"].str.contains(r'densenet161_CosFace_AdamW_0.001_cosine*')]
densenet161=densenet161[densenet161["epoch"]==99].dropna().tail(seed)
dpn2 = df_rank[df_rank["index"].str.contains(r'dpn107_MagFace_SGD_0.1_cosine*')]
dpn2=dpn2[dpn2["epoch"]==99].dropna().tail(seed)
dpns=df_rank[df_rank["index"].str.contains(r'dpn107_CosFace_SGD_0.1_cosine*')]
dpns=dpns[dpns["epoch"]==99].dropna().tail(seed)
model_000=df_rank[df_rank["index"].str.contains(r'Checkpoints_Edges_000*')]#.tail(5)
model_000=model_000[model_000["epoch"]==99].dropna().tail(seed)
model_010=df_rank[df_rank["index"].str.contains(r'Checkpoints_Edges_010*')]#.tail(5)
model_010=model_010[model_010["epoch"]==99].dropna().tail(seed)
model_680=df_rank[df_rank["index"].str.contains(r'Checkpoints_Edges_680*')]#.tail(5)
model_680=model_680[model_680["epoch"]==99].dropna().tail(seed)
model_040=df_rank[df_rank["index"].str.contains(r'Checkpoints_Edges_040*')]#.tail(5)
model_040=model_040[model_040["epoch"]==99].dropna().tail(seed)
df_rank_4 = pd.DataFrame()
df_rank_4["Accuracy_mean"]=[mobnet["Accuracy"].mean(),tnt["Accuracy"].mean(),rexnet["Accuracy"].mean(), ese["Accuracy"].mean(),densenet161["Accuracy"].mean(),dpn2["Accuracy"].mean(),dpns["Accuracy"].mean(), model_000["Accuracy"].mean(), model_010["Accuracy"].mean(), model_680["Accuracy"].mean()]#,model_040["Accuracy"].mean()]
df_rank_4["Accuracy_std"]=[sem(mobnet["Accuracy"]),sem(tnt["Accuracy"]),sem(rexnet["Accuracy"]),sem(ese["Accuracy"]),sem(densenet161["Accuracy"]),sem(dpn2["Accuracy"]),sem(dpns["Accuracy"]), sem(model_000["Accuracy"]), sem(model_010["Accuracy"]), sem(model_680["Accuracy"])]#,sem(model_040["Accuracy"])]
df_rank_4["Disparity_mean"]=[mobnet["Disparity"].mean(),tnt["Disparity"].mean(),rexnet["Disparity"].mean(),ese["Disparity"].mean(),densenet161["Disparity"].mean(),dpn2["Disparity"].mean(),dpns["Disparity"].mean(), model_000["Disparity"].mean(), model_010["Disparity"].mean(), model_680["Disparity"].mean()]#, model_040["Disparity"].mean()]
df_rank_4["Disparity_std"]=[sem(mobnet["Disparity"]),sem(tnt["Disparity"]),sem(rexnet["Disparity"]),sem(ese["Disparity"]),sem(densenet161["Disparity"]), sem(dpn2["Disparity"]),sem(dpns["Disparity"]), sem(model_000["Disparity"]), sem(model_010["Disparity"]), sem(model_680["Disparity"])]#,sem(model_040["Disparity"])]
df_rank_4["Model"]=["Mobilenet","TNT","Rexnet","Esevovnet","Densenet","Dpn_Magface","Dpn_Cosface", "SMAC", "SMAC", "SMAC"]
fig = px.scatter(df_rank_4, x='Accuracy_mean', y='Disparity_mean', error_x = "Accuracy_std", error_y = "Disparity_std", color='Model', template="simple_white",)
fig.update_layout(
    yaxis_title="Rank Disparity",
    xaxis_title = "Error",
    xaxis_range=[0,0.4]
)
fig.add_shape(type='line',
                x0=0,y0=0,x1=1,y1=0,
                line=dict(color='Red',),
                xref='x',yref='y'
)
fig.update_layout(
    xaxis_title="Error",
    yaxis_title="Rank Disparity",
    legend_title="Models",
    font=dict(
        family="Arial",
        size=14,
        color="Black"
    )
)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
p = np.array(preparePareto(df_rank_4[['Accuracy_mean','Disparity_mean']], False, False).dropna())
for x, y in zip(p[:-1], p[1:]):
    fig.add_shape(type='line',
                x0=x[0],y0=x[1],x1=y[0],y1=y[1],
                line=dict(color='gray',width=4),line_dash='dash',
                xref='x',yref='y')
fig.write_html("pareto_rank_disp_4.html")
fig.show()